In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/satislar.csv


In [2]:
#We import the necessary libraries
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
#We read data set
df = pd.read_csv('/kaggle/input/satislar.csv',sep=';',low_memory=False, header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,1145,2236,2020-05-01 08:29:49.000,11450001261903,426410,0,1,05.07.014,"Tıraş Bıçak ve Jiletleri,,"
1,1129,2156,2020-05-01 08:30:44.000,11290001181201,59010,0,"1,5",03.03.002,"Dana Eti Kg,"
2,1141,2216,2020-05-01 08:36:56.000,11410001201443,314720,0,1,02.02.004,"Meyve Suları Küçük Boy,,"
3,1141,2216,2020-05-01 08:36:56.000,11410001201443,180122,1,1,01.07.002,"Çikolata,,"
4,1141,2216,2020-05-01 08:36:56.000,11410001201443,162169,2,1,01.04.002,"Bisküvi Çeşitleri,,"


In [4]:
#We change the variable names
df.columns = ['BranchId', 'PosId', 'InvoiceDate', 'InvoiceNo','StockCode','Line','Quantity','CategoryCode','CategoryName']
df.head()

,BranchId,PosId,InvoiceDate,InvoiceNo,StockCode,Line,Quantity,CategoryCode,CategoryName
0,1145,2236,2020-05-01 08:29:49.000,11450001261903,426410,0,1,05.07.014,"Tıraş Bıçak ve Jiletleri,,"
1,1129,2156,2020-05-01 08:30:44.000,11290001181201,59010,0,"1,5",03.03.002,"Dana Eti Kg,"
2,1141,2216,2020-05-01 08:36:56.000,11410001201443,314720,0,1,02.02.004,"Meyve Suları Küçük Boy,,"
3,1141,2216,2020-05-01 08:36:56.000,11410001201443,180122,1,1,01.07.002,"Çikolata,,"
4,1141,2216,2020-05-01 08:36:56.000,11410001201443,162169,2,1,01.04.002,"Bisküvi Çeşitleri,,"


In [5]:
df.shape

(1048576, 9)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048576 entries, 0 to 1048575
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   BranchId      1048576 non-null  int64 
 1   PosId         1048576 non-null  int64 
 2   InvoiceDate   1048576 non-null  object
 3   InvoiceNo     1048576 non-null  int64 
 4   StockCode     1048576 non-null  int64 
 5   Line          1048576 non-null  int64 
 6   Quantity      1048576 non-null  object
 7   CategoryCode  974743 non-null   object
 8   CategoryName  1048576 non-null  object
dtypes: int64(5), object(4)
memory usage: 72.0+ MB


In [7]:
#We check for missing data
df.isnull().sum()

BranchId            0
PosId               0
InvoiceDate         0
InvoiceNo           0
StockCode           0
Line                0
Quantity            0
CategoryCode    73833
CategoryName        0
dtype: int64

In [8]:
#We check for missing data in CategoryCode
df[df["CategoryCode"].isnull()]

,BranchId,PosId,InvoiceDate,InvoiceNo,StockCode,Line,Quantity,CategoryCode,CategoryName
5,1141,2216,2020-05-01 08:37:43.000,11410001201444,998000,0,1,NaN,"TANIMSIZ,,"
13,1128,2151,2020-05-01 08:45:54.000,11280000880615,998000,5,2,NaN,"TANIMSIZ,,"
68,1130,2162,2020-05-01 08:48:23.000,11300010283056,998000,5,1,NaN,"TANIMSIZ,,"
82,1141,2216,2020-05-01 08:51:32.000,11410001201446,998000,2,1,NaN,"TANIMSIZ,,"
87,1148,2252,2020-05-01 08:48:07.000,11480001359474,998000,4,1,NaN,"TANIMSIZ,,"
...,...,...,...,...,...,...,...,...,...
1048515,1125,2137,2020-05-15 12:30:40.000,11250002395510,998000,13,2,NaN,"TANIMSIZ,,"
1048526,1125,2137,2020-05-15 12:36:25.000,11250002395514,557173,0,1,NaN,"TANIMSIZ,,"
1048527,1125,2137,2020-05-15 12:36:25.000,11250002395514,557173,1,1,NaN,"TANIMSIZ,,"
1048531,1125,2137,2020-05-15 12:38:12.000,11250002395516,998000,2,1,NaN,"TANIMSIZ,,"


In [9]:
#We can delete the CategoryId because CategoryName doesn't make sense.
df.dropna(subset= ["CategoryCode"],inplace= True)
df.shape

(974743, 9)

In [10]:
#How many unique values do I have
df.nunique()

BranchId            55
PosId               83
InvoiceDate     194210
InvoiceNo       256495
StockCode         6131
Line                97
Quantity          4521
CategoryCode       258
CategoryName       277
dtype: int64

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BranchId,974743.0,1.535722e+03,1.004966e+03,1.101000e+03,1.117000e+03,1.131000e+03,1.144000e+03,4.010000e+03
PosId,974743.0,2.454675e+03,6.997097e+02,2.016000e+03,2.097000e+03,2.167000e+03,2.236000e+03,4.021000e+03
InvoiceNo,974743.0,1.535723e+13,1.004966e+13,1.101000e+13,1.117000e+13,1.131000e+13,1.144000e+13,4.010000e+13
StockCode,974743.0,2.065353e+05,1.606901e+05,1.011000e+04,1.260050e+05,1.543620e+05,3.141530e+05,9.000674e+06
Line,974743.0,3.802201e+00,5.505346e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.000000e+00,9.600000e+01


In [12]:
# Stripping extra spaces in the description 
df['CategoryName'] = df['CategoryName'].str.strip(',')
df.head()

,BranchId,PosId,InvoiceDate,InvoiceNo,StockCode,Line,Quantity,CategoryCode,CategoryName
0,1145,2236,2020-05-01 08:29:49.000,11450001261903,426410,0,1,05.07.014,Tıraş Bıçak ve Jiletleri
1,1129,2156,2020-05-01 08:30:44.000,11290001181201,59010,0,"1,5",03.03.002,Dana Eti Kg
2,1141,2216,2020-05-01 08:36:56.000,11410001201443,314720,0,1,02.02.004,Meyve Suları Küçük Boy
3,1141,2216,2020-05-01 08:36:56.000,11410001201443,180122,1,1,01.07.002,Çikolata
4,1141,2216,2020-05-01 08:36:56.000,11410001201443,162169,2,1,01.04.002,Bisküvi Çeşitleri


In [13]:
pd.DataFrame(df["CategoryName"].value_counts(normalize=True)).head(100)

,CategoryName
Sebze,0.188234
Çikolata,0.052589
Sigara,0.049347
Meyve,0.037109
Bisküvi Çeşitleri,0.035389
...,...
Kakao,0.001452
Kırtasiye,0.001383
Pasta Kuru,0.001363
"Pisirme Torbası, Kagıdı",0.001350


In [14]:
#We change Quantity int
df["Quantity"] = df["Quantity"].astype("str")
df['Quantity'] = [x.replace(',', '.') for x in df['Quantity']]
df["Quantity"] = df["Quantity"].astype("float")
df["Quantity"] = df["Quantity"].astype("int")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 974743 entries, 0 to 1048575
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BranchId      974743 non-null  int64 
 1   PosId         974743 non-null  int64 
 2   InvoiceDate   974743 non-null  object
 3   InvoiceNo     974743 non-null  int64 
 4   StockCode     974743 non-null  int64 
 5   Line          974743 non-null  int64 
 6   Quantity      974743 non-null  int64 
 7   CategoryCode  974743 non-null  object
 8   CategoryName  974743 non-null  object
dtypes: int64(6), object(3)
memory usage: 74.4+ MB


In [15]:
branch_order = (df
          .groupby(['InvoiceNo', 'CategoryName'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [16]:
branch_order.head(20)

CategoryName,Agız Bakım Suyu,Ahsap Temizleyici,Ahsap Urunler,Aluminyum Folyo,Ampul Cesitleri,Arap Sabunu,Ayakkabı Boya Malzemeleri,Ayran,Ayçiçek Yağı,Ağda Malzemeleri,...,Çay Bardak,Çay Bitki,Çay Demlik,Çay Kg,Çikolata,Çocuk Bezi,Çorba,İrmik,Şalgam Suyu,Şampuan
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
11010001071064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010001071072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Encoding the datasets 
branch_encoded = branch_order.applymap(lambda x: 0 if x<=0 else 1) 
basket_branch = branch_encoded 

In [18]:
frq_items = apriori(basket_branch, min_support = 0.01, use_colnames = True)
frq_items

,support,itemsets
0,0.010608,(Ayran)
1,0.017182,(Ayçiçek Yağı)
2,0.018928,(Baharat Paketli)
3,0.067303,(Bisküvi Çeşitleri)
4,0.017248,(Bulaşık Sıvı Yıkama Deterjanı)
...,...,...
63,0.020332,"(Sebze, Sigara)"
64,0.014983,"(Sebze, Sut Sade)"
65,0.012924,"(Sebze, Yumurta)"
66,0.019049,"(Sebze, Çikolata)"


In [20]:
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.20) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(20) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Kraker),(Bisküvi Çeşitleri),0.021326,0.067303,0.011353,0.532358,7.909821,0.009918,1.994468
12,(Meyve),(Sebze),0.048157,0.230854,0.021178,0.439767,1.904953,0.010061,1.372903
0,(Kek ve Pasta),(Bisküvi Çeşitleri),0.035178,0.067303,0.014460,0.411061,6.107571,0.012093,1.583689
9,(Kek ve Pasta),(Çikolata),0.035178,0.091249,0.012636,0.359193,3.936392,0.009426,1.418135
5,(Bisküvi Çeşitleri),(Çikolata),0.067303,0.091249,0.021533,0.319933,3.506138,0.015391,1.336266
7,(Dana Eti Kg),(Sebze),0.040480,0.230854,0.012760,0.315227,1.365479,0.003415,1.123212
15,(Yumurta),(Sebze),0.042075,0.230854,0.012924,0.307172,1.330587,0.003211,1.110154
6,(Cips),(Çikolata),0.044964,0.091249,0.012647,0.281280,3.082541,0.008544,1.264401
14,(Sut Sade),(Sebze),0.055147,0.230854,0.014983,0.271686,1.176872,0.002252,1.056063
13,(Piliç Eti Kg),(Sebze),0.046820,0.230854,0.012534,0.267716,1.159674,0.001726,1.050338
